In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline
import pandas as pd
import spacy
import torch
import os
from sklearn.model_selection import train_test_split

# Set Hugging Face API key
os.environ["HUGGINGFACE_TOKEN"] = "your_correct_api_key_here"

# Load SpaCy NER
nlp = spacy.load("en_core_web_sm")

# Fine-tuning the DeBERTa V3 model
class ModelTrainer:
    def __init__(self, csv_path):
        self.csv_path = csv_path
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
        self.model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-base", num_labels=4)

    def load_data(self):
        df = pd.read_csv(self.csv_path)
        train_texts, val_texts, train_labels, val_labels = train_test_split(
            df['text'].tolist(), df['label'].tolist(), test_size=0.2)
        train_encodings = self.tokenizer(train_texts, truncation=True, padding=True)
        val_encodings = self.tokenizer(val_texts, truncation=True, padding=True)
        label_map = {"GENERATE_DOCUMENT": 0, "UPDATE_RULES": 1, "QUERY_INFORMATION": 2, "DELETE_DOCUMENT": 3}
        train_labels = [label_map[label] for label in train_labels]
        val_labels = [label_map[label] for label in val_labels]
        return train_encodings, val_encodings, train_labels, val_labels

    def train_model(self):
        train_encodings, val_encodings, train_labels, val_labels = self.load_data()
        train_dataset = IntentDataset(train_encodings, train_labels)
        val_dataset = IntentDataset(val_encodings, val_labels)
        training_args = TrainingArguments(
            output_dir="./results",
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=3,
            weight_decay=0.01
        )
        trainer = Trainer(model=self.model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset)
        trainer.train()
        self.model.save_pretrained("./fine_tuned_deberta")
        self.tokenizer.save_pretrained("./fine_tuned_deberta")

# Custom Dataset for Training
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# Preprocessing: Tokenization
class Preprocessor:
    def tokenize(self, text):
        return text.split()

# Intent Detection using Fine-Tuned DeBERTa V3
class IntentDetector:
    def __init__(self):
        self.classifier = pipeline("text-classification", model="./fine_tuned_deberta", tokenizer="./fine_tuned_deberta")

    def detect_intent(self, text):
        result = self.classifier(text)
        label = result[0]['label']
        score = result[0]['score']
        return label, score


In [ ]:
# Main Execution
if __name__ == "__main__":
    trainer = ModelTrainer("intent_dataset.csv")
    trainer.train_model()
    intent_detector = IntentDetector()
    text = input("Enter your command: ")
    raw_label, confidence = intent_detector.detect_intent(text)
    print(f"Detected Intent: {raw_label}, Confidence: {confidence:.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rajendraskyofy (rajendraskyofy-kafka-technologies) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Device set to use cuda:0


Enter your command: create a document which contains following points 
Detected Intent: LABEL_0, Confidence: 0.60


In [ ]:
#{"GENERATE_DOCUMENT": 0, "UPDATE_RULES": 1, "QUERY_INFORMATION": 2, "DELETE_DOCUMENT": 3}